<a href="https://colab.research.google.com/github/Jaekyung-Cho/DeeplearningTransformMatrix/blob/master/firstTry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# For starting, import libraries ###########
#import Tensorflow 2.0

%tensorflow_version=2.x
import tensorflow as tf

!pip install mitdeeplearning
import mitdeeplearning as mdl

import matplotlib.pyplot as plt
import numpy as np
import random
from tqdm import tqdm

assert len(tf.config.list_physical_devices('GPU')) > 0

Unknown TensorFlow version: =2.x
Currently selected TF version: 2.x
Available versions:
 * 1.x
 * 2.x
     |████████████████████████████████| 2.1MB 3.5MB/s 
  Created wheel for mitdeeplearning: filename=mitdeeplearning-0.1.2-cp36-none-any.whl size=2114586 sha256=7d7272d35bb0a1f8084b8dd722120db1e483c15ea704e2f0c77638d2ec43866c
  Stored in directory: /root/.cache/pip/wheels/27/e1/73/5f01c787621d8a3c857f59876c79e304b9b64db9ff5bd61b74
Successfully built mitdeeplearning


In [0]:
import math

def RandHomoTrans():
    T = [[0 for col in range(4)] for row in range(4)]
    
    theta = np.random.rand(3)*2*math.pi
    R1 = np.asmatrix([[math.cos(theta[0]), -math.sin(theta[0]),0],
         [math.sin(theta[0]), math.cos(theta[0]),0],
         [0,0,1]])
    
    R2 = np.asmatrix([[math.cos(theta[1]),0, -math.sin(theta[1])],
         [0,1,0],
         [math.sin(theta[1]),0, math.cos(theta[1])]])
    
    R3 = np.asmatrix([[1,0,0],
         [0,math.cos(theta[2]), -math.sin(theta[2])],
         [0,math.cos(theta[2]), -math.sin(theta[2])]])
    

    R = np.asarray(R1*R2*R3)
    
    P = np.random.rand(3)
    
    for i in range(3):
        for j in range(3):
            T[i][j] = R[i][j]
    
    for i in range(3):
        T[i][3] = P[i]
        
    T[3][3] = 1
    
    return np.asarray(T)

In [0]:
#making training set

train_data = []
train_label = []
test_data = []
test_label = []

data_dim = 1000

for i in range(10000):
  
  T = RandHomoTrans()
  x = (np.random.rand(data_dim)-0.5)*5
  y = (np.random.rand(data_dim)-0.5)*5
  z = (np.random.rand(data_dim)-0.5)*5

  train_data_tmp = []
  train_data_tmp_T = []

  for j in range(data_dim):
    train_data_tmp.append(x[j])
    train_data_tmp.append(y[j])
    train_data_tmp.append(z[j])
    p = [[x[j]],
         [y[j]],
         [z[j]],
         [1]]

    p =np.asarray( np.asmatrix(T) * np.asmatrix(p) )

    train_data_tmp_T.append(p[0])
    train_data_tmp_T.append(p[1])
    train_data_tmp_T.append(p[2])

  if i%10==0:
    test_data.append(train_data_tmp+train_data_tmp_T)
    test_label.append(T[0:3][:].reshape(1,12)[0])

  else:
    train_data.append(train_data_tmp+train_data_tmp_T)
    train_label.append(T[0:3][:].reshape(1,12)[0])

In [8]:
import numpy as np
a=np.asarray([[1,2,3,4],[1,1,1,1],[1,1,1,1],[3,3,3,3]])
b=[5,6]
print(a[0:3][:].reshape(1,12)[0])

[1 2 3 4 1 1 1 1 1 1 1 1]


In [0]:
def build_fc_model():
  fc_model = tf.keras.Sequential([
                                  tf.keras.layers.Dense(1000, activation='relu', input_shape=(6000,)),
                                  tf.keras.layers.Dense(120, activation='relu'),
                                  tf.keras.layers.Dense(12, activation='linear')
  ])
  return fc_model

model = build_fc_model()

In [0]:
#FCN compile

# In case of Adam, learning rate should be lower than 1e-3
model.compile(optimizer = tf.keras.optimizers.Adam(lr = 1e-3),
              loss = tf.keras.losses.MSE,
              metrics=[tf.keras.metrics.RootMeanSquaredError()])


In [14]:
train_data_arr = np.asarray(train_data).astype(np.float32)
train_label_arr = np.asarray(train_label).astype(np.float32)
print(train_data_arr.shape)
print(train_label_arr.shape)

(9000, 6000)
(9000, 12)


In [15]:
#FCN training

Batch_size = 40
Epoch = 10

model.fit(train_data_arr, train_label_arr, batch_size = Batch_size, epochs = Epoch)

Epoch 1/10
225/225 [==============================] - 2s 7ms/step - loss: 0.3707 - root_mean_squared_error: 0.6089
Epoch 2/10
225/225 [==============================] - 1s 7ms/step - loss: 0.2942 - root_mean_squared_error: 0.5424
Epoch 3/10
225/225 [==============================] - 1s 7ms/step - loss: 0.2854 - root_mean_squared_error: 0.5343
Epoch 4/10
225/225 [==============================] - 1s 7ms/step - loss: 0.2886 - root_mean_squared_error: 0.5372
Epoch 5/10
225/225 [==============================] - 1s 7ms/step - loss: 0.2734 - root_mean_squared_error: 0.5229
Epoch 6/10
225/225 [==============================] - 2s 7ms/step - loss: 0.3041 - root_mean_squared_error: 0.5514
Epoch 7/10
225/225 [==============================] - 2s 7ms/step - loss: 0.2747 - root_mean_squared_error: 0.5241
Epoch 8/10
225/225 [==============================] - 1s 7ms/step - loss: 0.3364 - root_mean_squared_error: 0.5800
Epoch 9/10
225/225 [==============================] - 1s 7ms/step - loss: 0.2857

In [0]:
test_data_arr = np.asarray(test_data).astype(np.float32)
test_label_arr = np.asarray(test_label).astype(np.float32)

test_loss, test_acc = model.evaluate(test_data_arr, test_label_arr)

print(test_acc)